This script evaluates previously trained machine learning models and a climatology baseline on a preprocessed test dataset. It loads the models from disk, prepares input features and target variables, adds a climatology model for comparison, and computes performance metrics. The evaluation results are stored for further analysis.

In [1]:
import sys
import os
sys.path.append('/home/vgarcia/notebooks')

from models_functions import *

In [9]:
# Input file
experiment_name = "Optimize_ML_all_Orinoco"
trained_model_path = f"/home/vgarcia/experiments/ML_annual_new/{experiment_name}/"
preprocessing_path = "/data/dl20-data/climate_operational/Victor_data/preprocessed_datasets_ML_Orinoco"
models = ["Lasso", "HGBR"]

In [10]:
# Load test dataset
df = pd.read_csv(f"{preprocessing_path}/test/test.csv", index_col=0).reset_index()
target_cols = ["rx90p_anom", "pr_anom"]
y_test = df[target_cols]
X_test = df.drop(columns=target_cols + ["year"])

In [11]:
# load models
models_dict = {}
for model in models:
    models_dict[model] = joblib.load(f"{trained_model_path}{model}_model.pkl")

# create climatology model
clim_model = ClimatologyModel(time_group="annual")
clim_model.fit(X_test, None, target_lag_prefix = "_DJF")
models_dict["Climatology"] = clim_model

evaluate_models(X_test, y_test, models_dict, store_testing=True, out_path = trained_model_path, experiment_name=experiment_name)

---Testing (MSE, MSSS, MAE, R2)---
Lasso: [0.16632725 0.15179765], [-0.15743708 -0.10019775], [0.30690766 0.2918823 ], [0.17510734 0.21319719]
HGBR: [0.15633923 0.15078546], [-0.08793248 -0.09286162], [0.30439771 0.29303453], [0.22464251 0.21844359]
Climatology: [0.14370306 0.13797306], [0. 0.], [0.29067353 0.27921774], [0.28731101 0.28485328]
